In [211]:
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max.columns', 100)
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from calendar import monthrange 
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer

In [212]:
pd.set_option('display.width', 1000) 
pd.set_option ('display.max_columns' , 1000)
pd.set_option("display.max_rows", 1000)


In [213]:
train_data = pd.read_csv('Data Sets/flight_delays_train.csv')
y_train = train_data['dep_delayed_15min']
x_test = pd.read_csv('Data Sets/flight_delays_test.csv')
train_data['dep_delayed_15min'] = train_data['dep_delayed_15min'].map({'N':0, 'Y':1})
x_train = train_data.drop(columns=['dep_delayed_15min'])

In [214]:
Air_Ports_Codes_Names = pd.read_excel('Data Sets/Data for comp/Airports .xlsx')
Population_of_cities = pd.read_excel('Data Sets/Data for comp/Population .xlsx')
Top_100_airports = pd.read_excel('Data Sets/Data for comp/Top Airports.xlsx')
Climate_features = pd.read_excel('Data Sets/Data for comp/Climate features .xlsx')
Snow_data = pd.read_excel('Data Sets/Data for comp/Snow data.xlsx')
Air_line_codes = pd.read_excel('Data Sets/Data for comp/Airlines codes .xlsx')
Top_By_passengers_carried = pd.read_excel('Data Sets/Data for comp/Top By passengers carried .xlsx')
Top_By_fleet_size = pd.read_excel('Data Sets/Data for comp/Top By fleet size.xlsx')

In [215]:
States_Time_zones = pd.read_excel('Data Sets/Data for comp/Time Zones.xlsx')
Time_zones = pd.read_excel('Data Sets/Data for comp/Differences between zones.xlsx')
States_Time_zones['State'] = States_Time_zones['State'].apply(lambda x: x.split('(')[0])
States_Time_zones['Time Zone'] = States_Time_zones['Time Zone'].apply(lambda x: x.split('(')[1][:-1])

In [216]:
Codes = Air_line_codes['Passenger carrier:'].apply(lambda x: x.split(' ')[len(x.split(' ')) - 1])
Codes = Codes.apply(lambda x: x[1:len(x) - 1])
Air_lines = Air_line_codes['Passenger carrier:'].apply(lambda x: ' '.join(x.split(' ')[:len(x.split(' '))-1]))
Air_line_codes = pd.DataFrame({'Air line': Air_lines, 'Code':Codes})

In [217]:
def extract_name_city(Pandas_Series):
    Series_of_cities = Pandas_Series.apply(lambda x: x.split(',')).apply(lambda x: x[0])
    Series_of_States  = Pandas_Series.apply(lambda x: x.split(',')).apply(lambda x: x[1])
    list_of_cities = []
    for city in Series_of_cities:
        my_list = city.split(' ')
        if 'city' in my_list:
            my_list.remove('city')
        if 'town'in my_list:
            my_list.remove('town')
        if 'village' in my_list:
            my_list.remove('village')
        if 'CDP' in my_list:
            my_list.remove('CDP')
        if 'urban' in my_list:
            my_list.remove('urban')
        if 'county' in my_list:
            my_list.remove('county')
        if '(Ventura)' in my_list:
            my_list.remove('(Ventura)')
        if '' in my_list:
            my_list.remove('village')
        if '(balance)' in my_list:
            my_list.remove('(balance)')
        if my_list[len(my_list)-1] == 'government':
            my_list = my_list[0]
        list_of_cities.append(my_list)
    return pd.Series(list_of_cities).map(lambda x: ' '.join(x))

In [218]:
Population_of_cities1 = pd.read_excel('Data Sets/Data for comp/Population .xlsx')

In [219]:
Population_of_cities.head(1)

,Unnamed: 0,Geographic Area,Population
0,225.0,"Abilene city, Texas",122186


In [220]:
Population_of_cities['Geographic Area'] = extract_name_city(Population_of_cities['Geographic Area'])
Population_of_cities = Population_of_cities.drop(columns=['Unnamed: 0'])
Population_of_cities.rename(columns={'Geographic Area':'City'},inplace= True)
Population_of_cities.insert(loc = 1, column = 'State',
                            value = Population_of_cities1['Geographic Area'].apply(lambda x: x.split(',')[1]))
Population_of_cities['State'] = Population_of_cities['State'].apply(lambda x: x[1:])

In [221]:
New_x_train = pd.read_csv('Data Sets/Data for comp/Final delay data.csv')
New_x_train1 = pd.read_csv('Data Sets/Data for comp/Final delay data.csv')

In [222]:
New_x_train.head(1)

,Unnamed: 0,Month,DayofMonth,DayOfWeek,DepTime,Dep part of day,Distance,Season,Is it holyday,Is it weekend,Part of month,Land time,Land part of day,UniqueCarrier,Origin,Dest,Origin_City,State_x,Dest_City,State_y,Origin_Population,Dest_Population,AL_in_Top_fleet_size,AL_in_Top_passengers_carried,Origin airport in top,Dest airport in top,Avg ° C Temp State_x,Prec Mill State_x,Sun hours State_x,Clear days State_x,Humidity,Avg ° C Temp State_y,Prec Mill State_y,Sun hours State_y,Clear days State_y,Humidity State_y,Target
0,0,8,21,7,1934,Evening,732,Summer,Not Holyday,weekend,3.0,1952,Evening,AA,ATL,DFW,Atlanta Georgia,Georgia,Dallas Texas,Texas,491670.0,1342479.0,0,0,1,1,25.9,120.0,882.0,22.0,54.0,27.3,69.0,935.0,41.0,44.0,0


In [223]:
set(Air_Ports_Codes_Names['State']) - set(Population_of_cities['State'])

{nan}

In [224]:
set(Population_of_cities['State']) - set(Air_Ports_Codes_Names['State'])

set()